In [ ]:
import tkinter as tk
from tkinter import filedialog
import numpy as np
import matplotlib
matplotlib.use('TkAgg')  # Set the backend to TkAgg
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# Define global variables
file_path = None

def browse_file():
    global file_path
    file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    if file_path:
        with open(file_path, 'r') as file:
            data_text.delete(1.0, tk.END)  # Clear previous text
            data_text.insert(tk.END, file.read())  # Insert text from file

def make_plot():
    global file_path
    if file_path:
        data = np.loadtxt(file_path)
        longitude = data[:, 0]
        latitude = data[:, 1]
        gravity_anomaly = data[:, 2]  # Define gravity_anomaly as a global variable
        lon_grid = np.linspace(min(longitude), max(longitude), 100)
        lat_grid = np.linspace(min(latitude), max(latitude), 100)
        lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)
        gravity_interp = griddata((longitude, latitude), gravity_anomaly, (lon_mesh, lat_mesh), method='linear')
        
        # Create a new figure window
        plt.figure()
        plt.contourf(lon_mesh, lat_mesh, gravity_interp, cmap='seismic')  # Use 'seismic' colormap for wider range from red to blue
        plt.colorbar(label='Gravity Anomaly')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.title('Gravity Anomaly Meshgrid Map')
        plt.show()

def first_vertical_derivative():
    global file_path, gravity_anomaly_fd
    
    if file_path:
        # Load data from file
        data = np.loadtxt(file_path)
        longitude_fd = data[:, 0]
        latitude_fd = data[:, 1]
        gravity_anomaly_fd = data[:, 2]  # Define gravity_anomaly as a global variable
        
        # Define kx as an array of values
#         kx = np.arange(0.0018868424, len(gravity_anomaly_fd) * 0.0018868424, 0.0018868424)
        gravity_fft_fd = np.fft.fft(gravity_anomaly_fd)
        
#         k_x_max = 2*np.pi/(0.033333333*111000)
#         k_x_min = 2*np.pi/(16.033333333*111000)
#         k_x = np.arange(k_x_min,k_x_max,0.033333333)
        
        
#         k_y_max = 2*np.pi/(0.033333333*111000)
#         k_y_min = 2*np.pi/(35.033333333*111000)
#         k_y = np.arange(k_y_min,k_y_max,0.033333333)


#         k = []
#         for i in range(len(k_x)):
#             for j in range(len(k_y)):
#                 k.append(np.sqrt(k_x[i]*k_x[i] + k_y[j]*k_y[j]))
        ki = 2 * np.pi / (0.033333333 * 111000)
        k_y = []
        for i in range(1052):
            k_y.append((i + 1) * ki)  # Use append to add elements to the list

        k_x = []
        for i in range(145):
            k_x.append(ki + i * ki)  # Use append to add elements to the list

#print(k_x)
        k = []
        for i in range(len(k_x)):
            for j in range(len(k_y)):
                k.append(np.sqrt(k_x[i]*k_x[i] + k_y[j]*k_y[j]))
                
        
        gravity_fft_filtered_fd = np.zeros_like(gravity_fft_fd, dtype=np.complex128)

        for i in range(len(k)):
            filter_fd = (k[i])
            # Apply the filter in the frequency domain
            gravity_fft_filtered_fd[i] = gravity_fft_fd[i]*filter_fd
        
        # Perform inverse Fourier Transform to obtain the filtered gravity anomaly
        gravity_anomaly_filtered_fd = np.real(np.fft.ifft(gravity_fft_filtered_fd))
        
        # Interpolate the modified gravity anomaly onto a grid
        lon_grid_fd = np.linspace(min(longitude_fd), max(longitude_fd), 100)
        lat_grid_fd = np.linspace(min(latitude_fd), max(latitude_fd), 100)
        lon_mesh_fd, lat_mesh_fd = np.meshgrid(lon_grid_fd, lat_grid_fd)
        gravity_interp_fd = griddata((longitude_fd, latitude_fd), gravity_anomaly_filtered_fd, (lon_mesh_fd, lat_mesh_fd), method='nearest')
        
        # Plot contour map of the interpolated gravity anomaly
        plt.figure()
        plt.contourf(lon_mesh_fd, lat_mesh_fd, gravity_interp_fd, cmap='seismic')  # Use 'seismic' colormap for wider range from red to blue
        plt.colorbar(label='Gravity Anomaly')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.title('First Vertical Derivative')
        plt.show()    
 

def second_vertical_derivative():
    global file_path
    if file_path:
        #dz = 1000  # Upward continuation height in meters
        data = np.loadtxt(file_path)
        longitude_sd = data[:, 0]
        latitude_sd = data[:, 1]
        gravity_anomaly_sd = data[:, 2]  # Define gravity_anomaly as a global variable
    
        # Calculate the Fourier Transform of the gravity anomaly
        gravity_fft_sd = np.fft.fft(gravity_anomaly_sd)
    
#         k_x_max = 2*np.pi/(0.033333333*111000)
#         k_x_min = 2*np.pi/(16.033333333*111000)
#         k_x = np.arange(k_x_min,k_x_max,0.033333333)
        
        
#         k_y_max = 2*np.pi/(0.033333333*111000)
#         k_y_min = 2*np.pi/(35.033333333*111000)
#         k_y = np.arange(k_y_min,k_y_max,0.033333333)


#         k = []
#         for i in range(len(k_x)):
#             for j in range(len(k_y)):
#                 k.append(np.sqrt(k_x[i]*k_x[i] + k_y[j]*k_y[j]))
        ki = 2 * np.pi / (0.033333333 * 111000)
        k_y = []
        for i in range(1052):
            k_y.append((i + 1) * ki)  # Use append to add elements to the list

        k_x = []
        for i in range(145):
            k_x.append(ki + i * ki)  # Use append to add elements to the list

#print(k_x)
        k = []
        for i in range(len(k_x)):
            for j in range(len(k_y)):
                k.append(k_x[i]*k_x[i] + k_y[j]*k_y[j])
        gravity_fft_filtered_sd = np.zeros_like(gravity_fft_sd, dtype=np.complex128)
        
        for i in range(len(k)):
            filter_sd = (k[i])
            # Apply the filter in the frequency domain
            gravity_fft_filtered_sd[i] = gravity_fft_sd[i] *filter_sd
        gravity_anomaly_filtered_sd = np.real(np.fft.ifft(gravity_fft_filtered_sd))

            
        # Interpolate the modified gravity anomaly onto a grid
        lon_grid_sd = np.linspace(min(longitude_sd), max(longitude_sd), 100)
        lat_grid_sd = np.linspace(min(latitude_sd), max(latitude_sd), 100)
        lon_mesh_sd, lat_mesh_sd = np.meshgrid(lon_grid_sd, lat_grid_sd)
        gravity_interp_sd = griddata((longitude_sd, latitude_sd), gravity_anomaly_filtered_sd, (lon_mesh_sd, lat_mesh_sd), method='nearest')
        
        # Plot contour map of the interpolated gravity anomaly
        plt.figure()
        plt.contourf(lon_mesh_sd, lat_mesh_sd, gravity_interp_sd, cmap='seismic')  # Use 'seismic' colormap for wider range from red to blue
        plt.colorbar(label='Gravity Anomaly')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.title('Second Vertical Derivative')
        plt.show()

def upward_continuation_filter():
    global file_path
    if file_path:
        dz = 20 # Upward continuation height in meters

        
        data = np.loadtxt(file_path)
        longitude = data[:, 0]
        latitude = data[:, 1]
        gravity_anomaly_uc = data[:, 2]  # Define gravity_anomaly as a global variable
    
        # Calculate the Fourier Transform of the gravity anomaly
        gravity_fft_uc = np.fft.fft(gravity_anomaly_uc)
#         k_x_max = 2*np.pi/(0.033333333*111000)
#         k_x_min = 2*np.pi/(16.033333333*111000)
#         k_x = np.arange(k_x_min,k_x_max,0.033333333)
        
        
#         k_y_max = 2*np.pi/(0.033333333*111000)
#         k_y_min = 2*np.pi/(35.033333333*111000)
#         k_y = np.arange(k_y_min,k_y_max,0.033333333)


#         k = []
#         for i in range(len(k_x)):
#             for j in range(len(k_y)):
#                 k.append(np.sqrt(k_x[i]*k_x[i] + k_y[j]*k_y[j]))
        ki = 2 * np.pi / (0.033333333 * 111000)
        k_y = []
        for i in range(1052):
            k_y.append((i + 1) * ki)  # Use append to add elements to the list

        k_x = []
        for i in range(145):
            k_x.append(ki + i * ki)  # Use append to add elements to the list

#print(k_x)
        k = []
        for i in range(len(k_x)):
            for j in range(len(k_y)):
                k.append(-dz*(np.sqrt(k_x[i]*k_x[i] + k_y[j]*k_y[j])))
        gravity_fft_filtered_uc = np.zeros_like(gravity_fft_uc, dtype=np.complex128)

        for i in range(len(k)):
            filter_uc =np.exp( k[i] )
            # Apply the filter in the frequency domain
            gravity_fft_filtered_uc[i] = gravity_fft_uc[i] * filter_uc
        gravity_anomaly_filtered_uc = np.real(np.fft.ifft(gravity_fft_filtered_uc))

        # Interpolate the modified gravity anomaly onto a grid
        lon_grid = np.linspace(min(longitude), max(longitude), 100)
        lat_grid = np.linspace(min(latitude), max(latitude), 100)
        lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)
        gravity_interp_uc = griddata((longitude, latitude), gravity_anomaly_filtered_uc, (lon_mesh, lat_mesh), method='linear')
        
        # Plot contour map of the interpolated gravity anomaly
        plt.figure()
        plt.contourf(lon_mesh, lat_mesh, gravity_interp_uc, cmap='seismic')  # Use 'seismic' colormap for wider range from red to blue
        plt.colorbar(label='Gravity Anomaly')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.title('Upward Continued Gravity Anomaly')
        plt.show()
                
        
root = tk.Tk()
root.title("Text File Reader")

browse_button = tk.Button(root, text="Browse File", command=browse_file)
browse_button.pack(pady=10)

plot_contour_button = tk.Button(root, text="Plot Contours", command=make_plot)
plot_contour_button.pack(pady=5)

first_vertical_derivative_button = tk.Button(root, text="First Vertical Derivative", command=first_vertical_derivative)
first_vertical_derivative_button.pack(pady=5)

second_vertical_derivative_button = tk.Button(root, text="Second Vertical Derivative", command=second_vertical_derivative)
second_vertical_derivative_button.pack(pady=5)

upward_continuation_button = tk.Button(root, text="Upward Continuation", command=upward_continuation_filter)
upward_continuation_button.pack(pady=5)

data_text = tk.Text(root, wrap=tk.WORD, width=50, height=20)
data_text.pack()

root.mainloop()


In [ ]:
# #Analytic Filter

# import tkinter as tk
# from tkinter import filedialog
# import numpy as np
# import matplotlib
# matplotlib.use('TkAgg')  # Set the backend to TkAgg
# import matplotlib.pyplot as plt
# from scipy.interpolate import griddata

# file_path = None

# def browse_file():
#     global file_path
#     file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
#     if file_path:
#         with open(file_path, 'r') as file:
#             data_text.delete(1.0, tk.END)  # Clear previous text
#             data_text.insert(tk.END, file.read())  # Insert text from file


# def analytic_filter():
#     global file_path
#     if file_path:
        
#         data = np.loadtxt(file_path)
#         longitude_af = data[:, 0]
#         latitude_af = data[:, 1]
#         gravity_anomaly_af = data[:, 2]  # Define gravity_anomaly as a global variable
 
#         gravity_fft_af = np.fft.fft(gravity_anomaly_af)
    
#         freq=np.fft.fftfreq(len(gravity_fft_af))
        
#         for i in range(len(gravity_fft_af)):
#             if (gravity_anomaly_af(i)>0):
#                 gravity_anomaly_af(i)=2*gravity_anomaly_fft(i)
#             else:
#                 gravity_anomaly_af(i)=0
#         gravity_anomaly_filtered = np.real(np.fft.ifft(gravity_fft_filtered))
        
#         # Interpolate the modified gravity anomaly onto a grid
#         lon_grid = np.linspace(min(longitude), max(longitude), 100)
#         lat_grid = np.linspace(min(latitude), max(latitude), 100)
#         lon_mesh, lat_mesh = np.meshgrid(lon_grid, lat_grid)
#         gravity_interp = griddata((longitude, latitude), gravity_anomaly_filtered, (lon_mesh, lat_mesh), method='linear')
        
#         # Plot contour map of the interpolated gravity anomaly
#         plt.figure()
#         plt.contourf(lon_mesh, lat_mesh, gravity_interp, cmap='seismic')  # Use 'seismic' colormap for wider range from red to blue
#         plt.colorbar(label='Gravity Anomaly')
#         plt.xlabel('Longitude')
#         plt.ylabel('Latitude')
#         plt.title('Upward Continued Gravity Anomaly')
#         plt.show()
        
        